In [3]:
import numpy as np

from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

unscaled_inputs_all = raw_csv_data[:,]

targets_all = raw_csv_data[:,-1]

In [4]:
num_one_targets = int(np.sum(targets_all))

zero_targets_counter = 0

indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove,axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove,axis=0)

In [6]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [8]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [10]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count,np.sum(test_targets)/ test_samples_count)



1786.0 3579 0.4990220732048058
229.0 447 0.5123042505592841
222.0 448 0.4955357142857143


In [11]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets = train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets = validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs,targets=test_targets)

In [13]:
import numpy as np
import tensorflow as tf

In [14]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')

validation_inputs, validation_targets = npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')

test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [17]:
input_size = 10
output_size = 2

hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')    
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
         train_targets,
         batch_size=batch_size,
         epochs=max_epochs,
         callbacks=[early_stopping],
         validation_data=(validation_inputs,validation_targets),
         verbose=2)

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 0.4894 - accuracy: 0.8287 - val_loss: 0.2323 - val_accuracy: 0.9955
Epoch 2/100
3579/3579 - 0s - loss: 0.1284 - accuracy: 0.9947 - val_loss: 0.0490 - val_accuracy: 1.0000
Epoch 3/100
3579/3579 - 0s - loss: 0.0305 - accuracy: 0.9980 - val_loss: 0.0136 - val_accuracy: 1.0000
Epoch 4/100
3579/3579 - 0s - loss: 0.0105 - accuracy: 1.0000 - val_loss: 0.0058 - val_accuracy: 1.0000
Epoch 5/100
3579/3579 - 0s - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 6/100
3579/3579 - 0s - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 7/100
3579/3579 - 0s - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 8/100
3579/3579 - 0s - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 9/100
3579/3579 - 0s - loss: 0.0011 - accuracy: 1.0000 - val_loss: 8.9014e-04 - val_accuracy: 1.0000
Epoch 10/100
3579/3579

Epoch 75/100
3579/3579 - 0s - loss: 1.0379e-05 - accuracy: 1.0000 - val_loss: 1.0935e-05 - val_accuracy: 1.0000
Epoch 76/100
3579/3579 - 0s - loss: 1.0052e-05 - accuracy: 1.0000 - val_loss: 1.0618e-05 - val_accuracy: 1.0000
Epoch 77/100
3579/3579 - 0s - loss: 9.7286e-06 - accuracy: 1.0000 - val_loss: 1.0278e-05 - val_accuracy: 1.0000
Epoch 78/100
3579/3579 - 0s - loss: 9.4183e-06 - accuracy: 1.0000 - val_loss: 9.9679e-06 - val_accuracy: 1.0000
Epoch 79/100
3579/3579 - 0s - loss: 9.1261e-06 - accuracy: 1.0000 - val_loss: 9.6687e-06 - val_accuracy: 1.0000
Epoch 80/100
3579/3579 - 0s - loss: 8.8391e-06 - accuracy: 1.0000 - val_loss: 9.3647e-06 - val_accuracy: 1.0000
Epoch 81/100
3579/3579 - 0s - loss: 8.5658e-06 - accuracy: 1.0000 - val_loss: 9.0908e-06 - val_accuracy: 1.0000
Epoch 82/100
3579/3579 - 0s - loss: 8.3033e-06 - accuracy: 1.0000 - val_loss: 8.8292e-06 - val_accuracy: 1.0000
Epoch 83/100
3579/3579 - 0s - loss: 8.0530e-06 - accuracy: 1.0000 - val_loss: 8.5628e-06 - val_accuracy:

In [18]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets)

448/448 [==============================] - 0s 225us/sample - loss: 5.5400e-06 - accuracy: 1.0000


In [19]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.00. Test accuracy: 100.00%
